# Prediction visualization notebook

In [1]:
from dataLoader import DataLoader, compute_euclidean_distance, prepare_input_img
from network import create_model
import os
import numpy as np
from custom_KFold import MyKFold
from block_matching_utils import find_template_pixel, global_template_search
from PIL import Image
import pandas as pd
from tensorflow import keras
import logging
import skimage
import matplotlib.pyplot as plt
from utils import *
from global_tracking import *

In [2]:
np.random.seed(seed=42)
exp_name = 'exp_80_50_128_se60'
params_dict = {'dropout_rate': 0.5, 'n_epochs': 5,
               'h3': 0, 'embed_size': 128, 'width': 80, 'search_w': 60}

# ============ DATA AND SAVING DIRS SETUP ========== #
data_dir = os.getenv('DATA_PATH')
test_dir = os.getenv('TEST_PATH')
exp_dir = os.getenv('EXP_PATH')
checkpoint_dir = os.path.join(exp_dir, exp_name)
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
params_dict = get_default_params(params_dict)

## Restore trained model

In [3]:
model = create_model(params_dict['width']+1,
                     params_dict['h1'],
                     params_dict['h2'],
                     params_dict['h3'],
                     embed_size=params_dict['embed_size'],
                     drop_out_rate=params_dict['dropout_rate'],
                     use_batch_norm=params_dict['use_batchnorm'])
model.load_weights(os.path.join(checkpoint_dir, 'model.h5'))

(?, 81, 81, 1)


### Get test folder

In [ ]:
# ['ETH-04-1' 'MED-01-1' 'CIL-01' 'MED-02-2']
testfolder = 'ETH-04-1' 
resolution_df = pd.read_csv(os.path.join(data_dir, 'resolution.csv'),
                                         sep=',\s+',
                                         decimal='.')
res_x, res_y = resolution_df.loc[resolution_df['scan']
                == testfolder, ['res_x', 'res_y']].values[0]
testfolder = os.path.join(data_dir, 'ETH-04-1')

/Applications/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """


### Predict on all features and save predictions

In [ ]:
predict_testfolder(testfolder, data_dir, res_x, res_y, model, params_dict)

['ETH-04-1_1.txt', 'ETH-04-1_2.txt']


/Applications/miniconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Applications/miniconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


100
200
300


## Visualize prediction for one feature

In [ ]:
img_dir = os.path.join(data_dir, testfolder, 'Data')
annotation_dir = os.path.join(data_dir, testfolder, 'Annotation')
feat = 'ETH-04-1_1.txt'
pred_feat1 = pd.read_csv(os.path.join(checkpoint_dir, feat),
                                         sep=',\s+',
                                         decimal='.')
true_feat1 = pd.read_csv(os.path.join(annotation_dir, feat),
                                         sep=',\s+',
                                         decimal='.')

In [ ]:

i = 100
try:
    img = np.asarray(Image.open(os.path.join(img_dir, '{:4d}.png'.format(i))))
except FileNotFoundError:
    img = np.asarray(Image.open(os.path.join(img_dir, '{:5d}.png'.format(i))))
plt.imshow(img)
plt.scatter(pred_feat1.loc[pred_feat1.id==i, c1], pred_feat1.loc[pred_feat1.id==i, c2])
plt.show()

In [ ]:
plt.plot(pred_feat1.id, pred_feat1.c1, 'b')
plt.show()
plt.plot(true_feat1.id, true_feat1.c1, 'r*')
plt.show()

In [ ]:
plt.plot(pred_feat1.id, pred_feat1.c2)
plt.show()